# Temporal water level changes

This notebook will draw graphs for temporal water level changes.
My idea is drawing separate graph for reference ground track (RFT). There are 7th cycles so far.
I will plot the water level of each cycle.

Steps are:
1. sepeate files by RFT
2. draw all cycless for each RFT

## Setting

Codes for starting this notebook from the previous notebook.

In [ ]:
import glob
import os
import pandas as pd
from pathlib import Path
import h5py
import re
import numpy as np
import matplotlib.pyplot as plt
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
##### load files
## set the directory
data_home = Path('/home/jovyan/ICESat_water_level/extraction/download/')
## list them up and check them
files= list(data_home.glob('*.h5'))
## choose a file as an example

##### function of reading alt13
def alt13_to_df(filename, beam):    
    f = h5py.File(filename, 'r')
    f_beam = f[beam]
    lat = f_beam['segment_lat'][:]
    long = f_beam['segment_lon'][:]
    ws = f_beam['ht_water_surf'][:]
    ws_sd = f_beam['stdev_water_surf'][:]
    ws_err = f_beam['err_ht_water_surf'][:]
    ortho = f_beam['ht_ortho'][:]
    wd = f_beam['water_depth'][:]
    alt13_df = pd.DataFrame({'Latitude':lat,'Longitude':long,'SurfaceH':ws,
                            'SH_SD':ws_sd, 'SH_error':ws_err,'OrthoH':ortho,
                            'WaterD':wd})
    return alt13_df

### We set 'gt2l' as a beam of example for tracking
D_dict={}
error_count=0
for ff in files:
    try:
        D_dict[ff]=alt13_to_df(ff, 'gt2l')
    except KeyError as e:
        ##print(f'file {ff} encountered error {e}')
        error_count += 1
print(f"read {len(D_dict)} data files of which {error_count} gave errors")

In [ ]:
### bounds of Tonle Sap Lake
sp_ex = [103.643, 104.667, 12.375, 13.287]

The file structure is:
- Examle: ATL13_2019031211225_09690201_003_01.h5
- `ATL13` = product
- `2019` = year
- `0312` = data
- `11225` = time
- `0969` = RGT
- `02` = cycle
- `01` = region
- `003` = data release
- `01` = version

Get the unique RGT: {'0529', '0025', '0901', '0017', '0085', '0969', '0961', '0461', '0521'}

In [ ]:
rgt_l = []
for dd in range(0,len(list(D_dict))):
    list_dd = str(list(D_dict)[dd])
    rgt_inf = list_dd.split("_")[5][0:4]
    rgt_l.append(rgt_inf)
print(rgt_l)    
uniq_l = set(rgt_l)
print(uniq_l)
    

Separating groups

In [ ]:
### new dictioray
new_dict = {}
for dd in range(0,len(list(D_dict))):
    list_dd = str(list(D_dict)[dd])
    rgt_inf = list_dd.split("_")[5]
    new_dict.update({rgt_inf: list(D_dict.values())[dd]})
print(new_dict)

select RGT "0961" for mapping

In [ ]:
##print(new_dict.keys())

In [ ]:
## 0961
list_0969 = [4,8,12,23,33,36]
list_0961 = [2,7,18,25,28,35]
list_0461 = [0,3,22,24,29]

Function for plotting photons

In [ ]:
import h5py
import numpy as np
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib as mpl        
import cartopy.io.img_tiles as cimgt

In [ ]:
def plot_photons (se_list, in_dic_key, in_dic_cal, waterL, title, bound):
    test_dict = {}
    for i in se_list:
        listed = str(list(in_dic_key.keys())[i])[4:6]
        test_dict.update({listed:list(in_dic_cal.values())[i]})
        
    fig=plt.figure(figsize=(6,4))
    ax = fig.add_subplot(111)
    for ii in list(test_dict):
        ax.plot(test_dict[ii]['Longitude'],test_dict[ii][waterL],markersize=0.25, label=ii)

    h_leg=ax.legend()
    plt.title(title)
    ax.set_xlabel('Longitude')
    ax.set_ylabel(waterL)
    plt.show()
    ## function for mapping the water level 
    def ic2_map_wl2(dicts, bound, water_levels):
        fig = plt.figure(figsize=(6,4))
        ax = plt.axes(projection=ccrs.PlateCarree())
        ax.set_extent(bound, crs=ccrs.PlateCarree())
        for i in dicts:
            plt.scatter(dicts[i]['Longitude'], dicts[i]['Latitude'], s=10, c=dicts[i][water_levels], alpha=.7, transform=ccrs.PlateCarree(), cmap='terrain')
        plt.colorbar(fraction=0.0320, pad=0.02, label='Elevation (m)')
        ##load Google Sat.Map
        #request = cimgt.GoogleTiles(style='satellite')
        request = cimgt.Stamen('terrain-background')
        ax.add_image(request, 7)
        plt.title(title)
    ic2_map_wl2(test_dict,bound,waterL)

In [ ]:
plot_photons(list_0969,new_dict,D_dict,'OrthoH','OrthoH_0969',sp_ex)

In [ ]:
plot_photons(list_0969,new_dict,D_dict,'SurfaceH','0969',sp_ex)

In [ ]:
plot_photons(list_0961,new_dict,D_dict,'SurfaceH','0961',sp_ex)

## Need to do
1. Which water level among surface water height and orthometric height is chose
2. There are so many gaps. This is because of different locations despite of same rgt, according to the map. What can we do?
    - Need to do additional literature reviews for tracking temporal water levels using ICESat1/2
    - Need to find futher official documents.